In [1]:
import gc
import os
import pickle
import random
import shutil

import numpy as np
import pandas as pd
import tensorflow as tf
from keras import losses, optimizers, metrics, callbacks

import SpeechModels

In [2]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
SEED = 123
N_CLASS = 12
MAX_EPOCHS = 200

In [4]:
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

## Load data

In [5]:
train_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/train",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=True,
    seed=SEED
)

val_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/val",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

test_ds = tf.keras.utils.audio_dataset_from_directory(
    directory="data/test",
    batch_size=512,
    output_sequence_length=16000,
    shuffle=False
)

label_names = np.array(train_ds.class_names)
print("label names:", label_names)

Found 45586 files belonging to 12 classes.
Found 6513 files belonging to 12 classes.
Found 13024 files belonging to 12 classes.
label names: ['down' 'go' 'left' 'no' 'off' 'on' 'right' 'silence' 'stop' 'unknown'
 'up' 'yes']


In [6]:
def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels


train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)
test_ds = test_ds.map(squeeze, tf.data.AUTOTUNE)

## Model from article

In [7]:
model = SpeechModels.AttRNNSpeechModel(N_CLASS, samplingrate=16000, inputLength=None)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None)]       0           []                               
                                                                                                  
 normalized_spectrogram_model (  (None, None, 80)    0           ['input[0][0]']                  
 Functional)                                                                                      
                                                                                                  
 tf.expand_dims (TFOpLambda)    (None, None, 80, 1)  0           ['normalized_spectrogram_model[0]
                                                                 [0]']                            
                                                                                              

In [8]:
weights = {
    0: 27.606189,
    1: 27.454890,
    2: 27.676583,
    3: 27.420211,
    4: 27.629614,
    5: 27.512886,
    6: 27.512886,
    7: 161.997512,
    8: 27.362605,
    9: 1.586856,
    10: 27.420211,
    11: 27.397139
}

In [9]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),
    loss=losses.SparseCategoricalCrossentropy(),
    metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
)

early_stopping = callbacks.EarlyStopping(
    monitor='val_sparse_categorical_accuracy',
    min_delta=0,
    patience=5,
    verbose=0,
    mode='max',
    baseline=None,
    restore_best_weights=True
)

reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_sparse_categorical_accuracy', factor=0.5, patience=3,
                                        min_lr=0.00001, verbose=1)

In [10]:
history = model.fit(
    train_ds,
    epochs=2,
    validation_data=val_ds,
    shuffle=True,
    callbacks=[early_stopping, reduce_lr],
    class_weight=weights
)

Epoch 1/2
90/90 [==============================] - 33s 185ms/step - loss: 17.1710 - sparse_categorical_accuracy: 0.3431 - sparse_categorical_crossentropy: 1.8149 - val_loss: 1.3042 - val_sparse_categorical_accuracy: 0.6059 - val_sparse_categorical_crossentropy: 1.3042 - lr: 0.0010
Epoch 2/2
90/90 [==============================] - 16s 170ms/step - loss: 5.6544 - sparse_categorical_accuracy: 0.7021 - sparse_categorical_crossentropy: 0.8228 - val_loss: 0.9593 - val_sparse_categorical_accuracy: 0.6670 - val_sparse_categorical_crossentropy: 0.9593 - lr: 0.0010


## Experiments

Training will be repeated 5 times with different weights initialization.

In [11]:
EXPERIMENT_NAME = "article_net_weighted"
if os.path.exists(EXPERIMENT_NAME):
    shutil.rmtree(EXPERIMENT_NAME)
    os.mkdir(EXPERIMENT_NAME)
else:
    os.mkdir(EXPERIMENT_NAME)

In [12]:
TRAINING_SEEDS = list(range(5))
results = []
for seed in TRAINING_SEEDS:
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

    model = SpeechModels.AttRNNSpeechModel(N_CLASS, samplingrate=16000, inputLength=None)

    model.compile(
        optimizer=optimizers.Adam(learning_rate=0.01),
        loss=losses.SparseCategoricalCrossentropy(),
        metrics=[metrics.SparseCategoricalAccuracy(), metrics.SparseCategoricalCrossentropy()]
    )

    history = model.fit(
        train_ds,
        epochs=MAX_EPOCHS,
        validation_data=val_ds,
        shuffle=True,
        callbacks=[early_stopping, reduce_lr],
        class_weight=weights
    )

    with open(os.path.join(EXPERIMENT_NAME, f"history_{seed}.pkl"), "wb") as file:
        pickle.dump(history.history, file)

    eval_results = model.evaluate(test_ds)

    predictions = model.predict(test_ds)
    with open(os.path.join(EXPERIMENT_NAME, f"predictions_{seed}.pkl"), "wb") as file:
        pickle.dump(predictions, file)

    results += [{
        'seed': seed,
        'results': dict(zip(model.metrics_names, eval_results))
    }]
    gc.collect()

results = pd.DataFrame(results)
results = pd.concat([results.drop(["results"], axis=1), results["results"].apply(pd.Series)], axis=1)
results.to_csv(os.path.join(EXPERIMENT_NAME, 'results.csv'))

Epoch 1/200
90/90 [==============================] - 23s 191ms/step - loss: 18.2065 - sparse_categorical_accuracy: 0.2857 - sparse_categorical_crossentropy: 1.8730 - val_loss: 1.9502 - val_sparse_categorical_accuracy: 0.6114 - val_sparse_categorical_crossentropy: 1.9502 - lr: 0.0100
Epoch 2/200
90/90 [==============================] - 18s 186ms/step - loss: 7.3855 - sparse_categorical_accuracy: 0.6666 - sparse_categorical_crossentropy: 0.9683 - val_loss: 2.3031 - val_sparse_categorical_accuracy: 0.0539 - val_sparse_categorical_crossentropy: 2.3031 - lr: 0.0100
Epoch 3/200
90/90 [==============================] - 17s 181ms/step - loss: 4.3818 - sparse_categorical_accuracy: 0.7894 - sparse_categorical_crossentropy: 0.6230 - val_loss: 1.5949 - val_sparse_categorical_accuracy: 0.4526 - val_sparse_categorical_crossentropy: 1.5949 - lr: 0.0100
Epoch 4/200
89/90 [============================>.] - ETA: 0s - loss: 4.0786 - sparse_categorical_accuracy: 0.8175 - sparse_categorical_crossentropy: 0

In [13]:
results

,seed,loss,sparse_categorical_accuracy,sparse_categorical_crossentropy
0,0,0.205750,0.953778,0.205750
1,1,0.189699,0.954469,0.189699
2,2,0.230322,0.950783,0.230322
3,3,0.243537,0.950630,0.243537
4,4,0.217451,0.935580,0.217451
